# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply



import app



if __name__ == "__main__":

    
    app.application.run()



In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [10]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta

#завтрашная дата без часов минут
def tomorrow_str():
    tomorrow = datetime.now()+ timedelta(days=1)
    tomorrow_str = str(tomorrow.year)+str(tomorrow.month if tomorrow.month >= 10 else  '0'+str(tomorrow.month))+str(tomorrow.day if tomorrow.day >= 10 else  '0'+str(tomorrow.day)) 
    return tomorrow_str

now = datetime.now()

dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }


tomorrow_str = '20170907'
user_id = 13


conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)

response = {'status' : 200
            ,'report' : 'tomorrow_expense'
            ,'system_message' : 'No reports'
            ,'user_messages' : []
            }
###################
cur = conn.cursor()
#смотрим, какие данные завтра есть
cur.execute("SELECT * from public.transaction_plan where date_plan = '%(tomorrow_str)s'" % {'tomorrow_str' : tomorrow_str} )

#получаем данные
df_tomorrow_transaction_plan = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

if df_tomorrow_transaction_plan.shape[0] == 0:
    cur.close()
    #return response
    print (response)
else:
    cur.execute("select id , chat_id from public.user" )
    df_user = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    cur.close()

#подгатавливаем лист завтрашних плановых операций
tomorrow_messages = []
#делаем выборку из пользователей, по кому завтра будет операция
list_tomorrow_user_id = list(df_tomorrow_transaction_plan['user_id'].unique())
df_tomorrow_user_id = df_user[df_user.id.isin(list_tomorrow_user_id)][:]

#проходим по ним и заполняем листр завтрашних плановых операций
for i,row in df_tomorrow_user_id.iterrows():
    #получаем данные пользователя
    user_id = row.id
    chat_id = row.chat_id
    #получаем сообщения по нему
    interest_info = df_tomorrow_transaction_plan[(df_tomorrow_transaction_plan.user_id == user_id)][['transaction_type','transaction_name','summa']][:]
    list_interest_info = list(interest_info.T.to_dict('list').values())
    
    text = 'Привет! Напоминаю, завтра есть запланированные траты: \n\n'
    num = 1
    for transaction_plan in list_interest_info:
        transaction_name = transaction_plan[1]
        summa = int(transaction_plan[2])
        text += str(num) + ': ' + transaction_name + ', Сумма: ' +str(summa)  + ' руб.\n'
        
        num += 1
        
    text += '\nНе забудьте решить эти вопросы, а то мало ли...'
        
    
    
    
    #формируем итоговый словарь по пользователю
    user_dict_tomorrow_messages = {'user_id' : user_id
                                  ,'chat_id' : chat_id
                                  ,'message' : text}
    #добавляем инфо
    tomorrow_messages.append(user_dict_tomorrow_messages)
    

response['system_message'] = 'Have reports'
response['user_messages'] = tomorrow_messages

print ('ok')

ok


In [13]:
from methods import telegram_bot_methods
g_reply_markup_main = {'keyboard': [['/wallet'],['/report'],['/crypto_ETH_USD']], 'resize_keyboard': True, 'one_time_keyboard': False}
if 'int' not in str(type(r)):
    if r['system_message'] == 'Have reports':
        list_messages = r['user_messages']
        for message in list_messages:
            #отправляем сообщение
            print(message)
            chat_id = str(message['chat_id'])
            text = message['message']
            send_result = telegram_bot_methods.send_message(chat_id = chat_id, text = text, reply_markup = reply_markup_main)

{'user_id': 14, 'chat_id': 164299093, 'message': 'Привет! Напоминаю, завтра есть запланированные траты: \n\n1: купить газель, Сумма: 1000 руб.\n2: купить газель1, Сумма: 1001 руб.\n\nНе забудьте решить эти вопросы, а то мало ли...'}
{'user_id': 13, 'chat_id': 84723474, 'message': 'Привет! Напоминаю, завтра есть запланированные траты: \n\n1: Купить барбосу игрушек, Сумма: 500 руб.\n2: картошка1, Сумма: 123 руб.\n3: картошка2, Сумма: 234 руб.\n4: картошка3, Сумма: 345 руб.\n5: Позвони маме!, Сумма: 1 руб.\n\nНе забудьте решить эти вопросы, а то мало ли...'}


TypeError: Object of type 'int64' is not JSON serializable